In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By

In [ ]:
edge:object = webdriver.Edge(executable_path="./utils/msedgedriver.exe")

### Functions:

In [ ]:
def close_window(driver:object) -> None:
    button:str = "//a[@class='btn btn-subscription-light']"
    driver.find_element(by=By.XPATH, value=button).click()

In [ ]:
def choose_category(category:str, driver:object) -> None:
    driver.find_element(by=By.XPATH, value=category).click()

In [ ]:
def exit_ad(driver:object) -> None:
    ...

In [ ]:
def extract_description(driver:object) -> str:
    description:str = ""
    exit_ad(driver)
    return description

### Extraction:

In [ ]:
elements = driver.find_elements(by=By.XPATH, value="//div[@class='ann-subcat-listing']")

In [ ]:
prices_obs = driver.find_elements(by=By.XPATH, value="//div[@class='ann-ad-tile__price']")

## Main: